# Tools & Toolkits

https://blog.langchain.dev/tool-calling-with-langchain/

https://python.langchain.com/v0.2/docs/how_to/tools_builtin/



## 1. API wrapper

**Requires** !pip install -qU wikipedia

**Note**
Refer to the documentation for Wikipedia API wrapper.

https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool


import warnings 

warnings.filterwarnings('ignore')

In [ ]:
# https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html

api_wrapper = WikipediaAPIWrapper(doc_content_chars_max=50, top_k_results=2)


In [ ]:
query="Prompt engineering"

response = api_wrapper.load(query = query)

response

## 2. Tool

Create a tool with the Wikipedia wrapper

https://api.python.langchain.com/en/latest/tools/langchain_core.tools.Tool.html#langchain_core.tools.Tool

In [ ]:
# Added 
api_wrapper = WikipediaAPIWrapper(doc_content_chars_max=50, top_k_results=2)

# Tool is a Runnable which means that it has a function invoke
wiki_tool = Tool(
    name="wikipedia",
    description="useful for searching the wikipedia",
    func=api_wrapper.load
)

In [ ]:
# Test
query="Prompt engineering"

response = wiki_tool.invoke({"query": query})

response

### Use the built in tool class for Wikipedia

https://api.python.langchain.com/en/latest/tools/langchain_community.tools.wikipedia.tool.WikipediaQueryRun.html

In [14]:
from langchain.tools import WikipediaQueryRun

wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# You can override the tool specs e.g., name, description
# wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(),name="mywiki", description="overridden the description")

wikipedia_tool.verbose=True

print("name: ", wikipedia_tool.name)
print("description: ", wikipedia_tool.description)
print("args_schema: ", wikipedia_tool.args_schema)

response = wiki_tool.invoke({"query": query})

NameError: name 'WikipediaAPIWrapper' is not defined

### Use load_tools

https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.load_tools.load_tools.html

#### List of tool names
https://api.python.langchain.com/en/latest/_modules/langchain_community/agent_toolkits/load_tools.html#load_tools

In [ ]:
from langchain_community.agent_toolkits.load_tools import load_tools

tools_to_use = load_tools([
    "wikipedia",
    "arxiv",
    "read_file"
])

for tool in tools_to_use:
    print("-----------------")
    print("name: ", tool.name)
    print("description: ", tool.description)


## 3. Toolkit

https://python.langchain.com/v0.1/docs/integrations/toolkits/

**JsonSpec** & **JsonToolkit** classes

https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.json.toolkit.JsonToolkit.html#langchain_community.agent_toolkits.json.toolkit.JsonToolkit



In [ ]:
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec


In [ ]:
# JSON - dictionary
data = {
    "a": "value-1",
    "b": "value-2",
    "c": {
        "d": "value-3",
        "e": "value-4"
    }
}

json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

# Get list of tools
tools = json_toolkit.get_tools()

# Pass the tools to create an agent
# agent = create_agent_method(llm, tools, prompt)

In [ ]:
for tool in tools:
    print('-----------------')
    print(tool.name)
    print(tool.description)

In [ ]:
tools[0].invoke('data["c"]')

In [ ]:
# Notice that the input is a string NOT a dictionary
tools[1].invoke('data["c"]["d"]')

## 4. Custom tools

https://python.langchain.com/v0.2/docs/how_to/custom_tools/

This **@tool decorator** is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

Another way to do it is by using the **StructuredTool** class.

https://blog.langchain.dev/structured-tools/


#### @tool

In [1]:
from langchain_core.tools import tool

@tool
def multiply_tool(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of the two integers.

    Example:
        >>> multiply(2, 3)
        6
    """
    return a * b


In [2]:
print(multiply_tool.name)
print("============Description====================")
print(multiply_tool.description)
print("============Args schema====================")
print(multiply_tool.args_schema.__dict__['__fields__'])

multiply_tool
============Description====================
Multiplies two integers.

Args:
    a (int): The first integer.
    b (int): The second integer.

Returns:
    int: The product of the two integers.

Example:
    >>> multiply(2, 3)
    6
============Args schema====================
{'a': ModelField(name='a', type=int, required=True), 'b': ModelField(name='b', type=int, required=True)}


#### StructuredTool

In [3]:
def division_tool(a: int, b: int) -> int:
    """
    Divides two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer. It MUST != 0

    Returns:
        int: The divison of the two integers.

    Example:
        >>> division(2, 3)
        6
    """
    return a / b

In [4]:
from langchain.tools.base import StructuredTool

division_tool = StructuredTool.from_function(division_tool)

print(division_tool.name)
print("============Description====================")
print(division_tool.description)
print("============Args schema====================")
print(division_tool.args_schema.__dict__['__fields__'])

division_tool
============Description====================
Divides two integers.

Args:
    a (int): The first integer.
    b (int): The second integer. It MUST != 0

Returns:
    int: The divison of the two integers.

Example:
    >>> division(2, 3)
    6
============Args schema====================
{'a': ModelField(name='a', type=int, required=True), 'b': ModelField(name='b', type=int, required=True)}


## Exercise : FileManagementToolkit

Go through the documentation for the file management toolkit : https://api.python.langchain.com/en/latest/agent_toolkits/langchain_community.agent_toolkits.file_management.toolkit.FileManagementToolkit.html

1. Create a temporary directory as the *working_dir*
2. List the tools available in file management toolkit
3. Write a file (or 2) to the folder
4. List the files in the folder
5. Read content of a file

#### Tools available for selection:
['copy_file', 'file_delete', 'file_search', 'move_file', 'read_file', 'write_file', 'list_directory']

### 1. Create temp folder

In [5]:
import os

# CHANGE the path
temp_folder_path = "c:/temp/temp"

if not os.path.exists(temp_folder_path):
    os.makedirs(temp_folder_path)

### 2. List available tools

In [6]:
from langchain_community.agent_toolkits import FileManagementToolkit

fm_toolkit = FileManagementToolkit(root_dir=temp_folder_path)

# Toolkit class function to get the list of tools 
fm_toolkit.get_tools()

[CopyFileTool(root_dir='c:/temp/temp'),
 DeleteFileTool(root_dir='c:/temp/temp'),
 FileSearchTool(root_dir='c:/temp/temp'),
 MoveFileTool(root_dir='c:/temp/temp'),
 ReadFileTool(root_dir='c:/temp/temp'),
 WriteFileTool(root_dir='c:/temp/temp'),
 ListDirectoryTool(root_dir='c:/temp/temp')]

### 3. Write a file

In [10]:
fm_toolkit = FileManagementToolkit(root_dir=temp_folder_path, selected_tools=['write_file'])

# Get just the write tool
write_file_tool = FileManagementToolkit(
    root_dir=temp_folder_path,
    selected_tools=["write_file"],
).get_tools()[0]

file_name = "example.txt"
write_file_tool.invoke({"file_path": file_name, "text": "Hello World!"})

'File written successfully to example.txt.'

### 4. List the files

In [12]:
fm_toolkit = FileManagementToolkit(root_dir=temp_folder_path, selected_tools=['list_directory'])

# Get just the list directory tool
list_file_tool = FileManagementToolkit(
    root_dir=temp_folder_path,
    selected_tools=["list_directory"],
).get_tools()[0]


list_file_tool.invoke("")

'example.txt'

### 5. Read the content of a file

In [13]:
fm_toolkit = FileManagementToolkit(root_dir=temp_folder_path, selected_tools=['read_file'])

# Get just the list directory tool
read_file_tool = FileManagementToolkit(
    root_dir=temp_folder_path,
    selected_tools=["read_file"],
).get_tools()[0]

read_file_tool.invoke(file_name)

'Hello World!'